Modifying the isopropanol/isobutanol distillation example from *The Organic Chem Lab Survival Manual* to see if phase diagrams can be plotted programmatically.

In [ ]:
%matplotlib widget

In [ ]:
import math
import numpy as np

Ideal behavior is calculated from the boiling point of the components of the binary mixture, and their heats of vaporization. 

In [ ]:
bp_isopropanol = 355.4  # K (82.6 °C)
bp_isobutanol = 381.2  # K (107.9 °C)
H_isopropanol = 9515.73  # cal/mol
H_isobutanol = 10039.70  # cal/mol

Clausius-Clapeyron equation:

$$p=p°e^{\frac{-\Delta H}{R}(\frac{1}{T} - \frac{1}{T°})}$$

In [ ]:
def p_from_t(h, t, p0, t0):
    """Calculate the vapor pressure of a liquid at a given temperature, using the Clausius-Clapeyron equation.
    
    The most convenient references p0 and t0 are from the boiling point of the compound,
    e.g. 760 torr and 355.4 K for isopropanol.
    
    Parameters
    ----------
    h : float
        The heat of vaporization in cal/mol
    t : float
        The temperature in K
    p0 : float
        Reference vapor pressure
    t0 : float
        Reference temperature in K
        
    Returns
    -------
    float
        The vapor pressure at temperature t.
    """
    r = 1.987204  # cal/mol·K
    e = - (h/r) * ((1/t) - (1/t0))
    return p0 * math.exp(e)

In [ ]:
# p = p_from_t(H_isobutanol, 364.1, 760, bp_isobutanol)
# p

In [ ]:
def isobutanol_pressure(t):
    """Calculate the vapor pressure (torr) of isobutanol, given a temperature in K."""
    return p_from_t(H_isobutanol, t, 760, bp_isobutanol)

def isopropanol_pressure(t):
    """Calculate the vapor pressure (torr) of isopropanol, given a temperature in K."""
    return p_from_t(H_isopropanol, t, 760, bp_isopropanol)

From *OCLSM* p. 327:

$$X_A = \frac{P_{atm}-P°_B}{P°_A-P°_B}$$

In [ ]:
def chi_isopropanol_liquid(t):
    """Calculate the mol fraction of isopropanol in an isopropanol/isubutanol mixture, 
    given its boiling point in K.
    """
    p_isopropanol = isopropanol_pressure(t)
    p_isobutanol = isobutanol_pressure(t)
    return (760 - p_isobutanol) / (p_isopropanol - p_isobutanol)

def chi_isobutanol_liquid(t):
    return 1.0 - chi_isopropanol_liquid(t)

In [ ]:
chi_isopropanol_liquid(92 + 273.15)

In [ ]:
import matplotlib
import matplotlib.pyplot as plt

From *OCLSM p. 329:

$$X^{vapor}_A = X^{liquid}_AP^°_A/760$$

In [ ]:
def chi_isobutanol_vapor(t):
    """Calculate the mol fraction isobutanol in the vapor phase at a given temperature,
    given the temperature t in K.
    """
    chi_liquid = chi_isobutanol_liquid(t)
    p = isobutanol_pressure(t)
    return chi_liquid * p / 760

In [ ]:
# create y coordinates
t_k = np.arange(bp_isopropanol, bp_isobutanol, 0.1)  # K
t_c = t_k - 273.15  # °C

# create vectorized versions of chi functions,
# then apply them to t_k to generate x coordinates 
f_liquid = np.vectorize(chi_isobutanol_liquid)
f_vapor = np.vectorize(chi_isobutanol_vapor)
chi_liquid = f_liquid(t_k)
chi_vapor = f_vapor(t_k)

Before creating the animated gif, the basic plot is created and saved in PNG format. This can be used for student assignments.

In [ ]:
fig, ax = plt.subplots()
plt.xlim(0.0, 1.0)
plt.ylim(80.0, 110.0)
liquid_curve = ax.plot(chi_liquid, t_c, color='#0072b2')[0]
vapor_curve = ax.plot(chi_vapor, t_c, color='#d55e00')[0]

ax.set(xlabel='mol fraction isobutanol', ylabel='T (°C)',
       title='Phase Diagram for Isopropanol/Isobutanol')
# ax.grid()

# Show the major grid lines with dark grey lines
ax.grid(b=True, which='major', color='#666666', linestyle='-')

# Show the minor grid lines with very faint and almost transparent grey lines
ax.minorticks_on()
ax.xaxis.grid(b=True, which='minor', color='#999999', linestyle='-', alpha=0.2, markevery=0.02)
ax.yaxis.grid(b=True, which='minor', color='#999999', linestyle='-', alpha=0.2)

import matplotlib.ticker as ticker
x_major_spacing = 0.1
x_minor_spacing = 0.02
ax.xaxis.set_minor_locator(ticker.MultipleLocator(x_minor_spacing))
ax.xaxis.set_major_locator(ticker.MultipleLocator(x_major_spacing))

# uncomment to view plot in notebook
# note: if Cell -> Run All was used to run the notebook, 
# this will show as an animation instead of a static plot
# plt.show()

In [ ]:
# When you want to save the plot, uncomment the code below

# fig.set_size_inches(9, 5.5)
# plt.savefig('iPrOH_iBuOH_phase_diagram.png', dpi=300, orientation='landscape', transparent=False)

In [ ]:
print(liquid_curve)

In [ ]:
from scipy.optimize import fsolve

In [ ]:
def test_func(t):
    return chi_isobutanol_liquid(t) - 0.8

In [ ]:
# fsolve(test_func, 102.0) - 273.15

In [ ]:
def solve_t(fn, chi):
    """Solve for t where fn(t) = chi.
    
    Parameters
    ----------
    fn : function taking one numerical argument
    
    Returns
    -------
    [t] : [float] 
        Temperature in K. Should only have one answer. 
        TODO: consider returning t directly.
    """
    test_function = lambda t: fn(t) - chi
    # using 90 °C or 363.15 K as first guess for fsolve
    return fsolve(test_function, 363.15)

In [ ]:
# bp_80 = solve_t(chi_isobutanol_liquid, 0.8)
# bp_80 - 273.15

In [ ]:
# vapor_80 = chi_isobutanol_vapor(bp_80[0])
# vapor_80

In [ ]:
def theoretical_plate(chi_initial):
    """Return the bp of a mixture in °C and its vapor composition,
    given the composition of the liquid.
    
    Parameters
    ----------
    chi_inital : float
        the mol fraction of isobutanol in the mixture
    
    Returns
    -------
    float, float
        tuple of (T in °C, mol fraction isobutanol in vapor)
    """
    t = solve_t(chi_isobutanol_liquid, chi_initial)[0]
    chi_vapor = chi_isobutanol_vapor(t)
    return t - 273.15, chi_vapor 

In [ ]:
# Starting from an initial composition of 0.8 mol fraction isobutanol,
# calculate the t and chi data for the theoretical plates

chi0 = 0.8
t0, chi1 = theoretical_plate(chi0)
t1, chi2 = theoretical_plate(chi1)
t2, chi3 = theoretical_plate(chi2)
t3, chi4 = theoretical_plate(chi3)
t4, chi5 = theoretical_plate(chi4)
t5, chi6 = theoretical_plate(chi5)
t6, chi7 = theoretical_plate(chi6)

In [ ]:
# t, v = theoretical_plate(0.8)
# t_c = t - 273.15
# t_c, v

In [ ]:
# t2, v2 = theoretical_plate(v)
# t2_c = t2 - 273.15
# t2_c, v2

In [ ]:
# line segments for theoretical plates

vertical1 = ax.plot([chi0, chi0], [0, t0], color='#999999', linestyle='--')[0]
horizontal1 = ax.plot([chi0, chi1], [t0, t0], color='#999999', linestyle='--')[0]
vertical2 = ax.plot([chi1, chi1], [t0, t1], color='#999999', linestyle='--')[0]
horizontal2 = ax.plot([chi1, chi2], [t1, t1], color='#999999', linestyle='--')[0]
vertical3 = ax.plot([chi2, chi2], [t1, t2], color='#999999', linestyle='--')[0]
horizontal3 = ax.plot([chi2, chi3], [t2, t2], color='#999999', linestyle='--')[0]
vertical4 = ax.plot([chi3, chi3], [t2, t3], color='#999999', linestyle='--')[0]
horizontal4 = ax.plot([chi3, chi4], [t3, t3], color='#999999', linestyle='--')[0]
vertical5 = ax.plot([chi4, chi4], [t3, t4], color='#999999', linestyle='--')[0]
horizontal5 = ax.plot([chi4, chi5], [t4, t4], color='#999999', linestyle='--')[0]
vertical6 = ax.plot([chi5, chi5], [t4, t5], color='#999999', linestyle='--')[0]
horizontal6 = ax.plot([chi5, chi6], [t5, t5], color='#999999', linestyle='--')[0]
vertical7 = ax.plot([chi6, chi6], [t5, t6], color='#999999', linestyle='--')[0]


In [ ]:
# tp1 = ([0.8, 0.8, v, v], [0, t_c, t_c, 0])
# ax.plot(*tp1, color='black')
# # plt.show()

In [ ]:
# tp2 = ([v, v, v2, v2], [0, t2_c, t2_c, 0])
# ax.plot(*tp2, color='black')

In [ ]:
# points for bp of pure components
isopropanol_bp_point = plt.scatter(0, bp_isopropanol-273.15, 
                                   clip_on=False, color='#0072b2')
isobutanol_bp_point = plt.scatter(1, bp_isobutanol-273.15, 
                                  clip_on=False, color='#0072b2')

In [ ]:
# points along the liquid and vapor curves that intersect with
# the line segments
l0 = plt.scatter([chi0], [t0], color='#0072b2')
v0 = plt.scatter([chi1], [t0], color='#d55e00')
l1 = plt.scatter([chi1], [t1], color='#0072b2')
v1 = plt.scatter([chi2], [t1], color='#d55e00')
l2 = plt.scatter([chi2], [t2], color='#0072b2')
v2 = plt.scatter([chi3], [t2], color='#d55e00')
l3 = plt.scatter([chi3], [t3], color='#0072b2')




What follows are the annotations that will be added during the animation:

In [ ]:
a_isopropanol_bp = plt.annotate(
    'boiling point of isopropanol\n(mol fraction isobutanol = 0)',
    xy=(0, bp_isopropanol-273.15),
    xycoords='data',
    fontsize=12,
    xytext=(+30, -0),
    textcoords='offset points',
    arrowprops={
        'arrowstyle': '->'
    })

In [ ]:
a_isobutanol_bp = plt.annotate(
    'boiling point of isobutanol\n(mol fraction isobutanol = 1)',
    xy=(1, bp_isobutanol-273.15),
    xycoords='data',
    fontsize=12,
    xytext=(-200, -10),
    textcoords='offset points',
    arrowprops={
        'arrowstyle': '->'
    })

In [ ]:
a_liquid = plt.annotate(
    'boiling point of\niPrOH/iBuOH mixture',
    xy=(0.5, solve_t(chi_isobutanol_liquid, 0.5)[0]-273.15),
    xycoords='data',
    fontsize=12,
    xytext=(+30, -30),
    textcoords='offset points',
    arrowprops={
        'arrowstyle': '->'
    })

In [ ]:
a_vapor = plt.annotate(
    'composition of vapor\n at boiling point T',
    xy=(chi2, t1),
    xycoords='data',
    fontsize=12,
    xytext=(-130, +20),
    textcoords='offset points',
    arrowprops={
        'arrowstyle': '->'
    })

In [ ]:
a1 = plt.annotate(
    f'bp of\n80 mol%\nisobutanol\n= {t0:.1f} °C',
    xy=(0.8, t0),
    xycoords='data',
    fontsize=12,
    xytext=(+30, -30),
    textcoords='offset points',
    arrowprops={
        'arrowstyle': '->'
    })

In [ ]:
a2 = plt.annotate(
    f'mol% of vapor\nat {t0:.1f} °C\n= {chi1*100:.0f}%',
    xy=(chi1, t0),
    xycoords='data',
    fontsize=12,
    xytext=(-110, -10),
    textcoords='offset points',
    arrowprops={
        'arrowstyle': '->'
    })

In [ ]:
a3 = plt.annotate(
    f'cooling these vapors\ngives a condensate\nthat is {chi1*100:.0f}% iBuOH',
    xy=(chi1, t1),
    xycoords='data',
    fontsize=12,
    xytext=(+30, -30),
    textcoords='offset points',
    arrowprops={
        'arrowstyle': '->'
    })

In [ ]:
a4 = plt.annotate(
    f'i.e. the amount of\nthe more volatile\niPrOH has increased\nfrom 20% to {(1-chi1)*100:.0f}%',
    xy=(chi1, t1),
    xycoords='data',
    fontsize=12,
    xytext=(+30, -30),
    textcoords='offset points',
    arrowprops={
        'arrowstyle': '->'
    })

In [ ]:
a5 = plt.annotate(
    'a single\nvaporize/condense\ncycle is called a\n' + r'$\bf{theoretical\ plate}$',
    xy=(chi1, t1),
    xycoords='data',
    fontsize=12,
    xytext=(+30, -30),
    textcoords='offset points',
    arrowprops={
        'arrowstyle': '->'
    })

In [ ]:
a6 = plt.annotate(
    '''So, if a simple distillation\n'''
    '''apparatus has an\n'''
    '''efficiency of ~ 1\n'''
    '''theoretical plate,\n'''
    '''the distillate will only be\n'''
    '''slightly enriched in iPrOH.''',
    xy=(chi1, t1),
    xycoords='data',
    fontsize=12,
    xytext=(+30, -30),
    textcoords='offset points',
    arrowprops={
        'arrowstyle': '->'
    })

In [ ]:
a7 = plt.annotate(
    '''In fractional distillation,\n'''
    '''multiple evaporate/condense\n'''
    '''steps occur before the\n'''
    '''vapors reach the condenser.\n''',
    xy=(chi2, t1),
    xycoords='data',
    fontsize=12,
    xytext=(-130, +15),
    textcoords='offset points',
    arrowprops={
        'arrowstyle': '->'
    })

In [ ]:
a8 = plt.annotate(
    '''An efficiency of 2\n'''
    '''theoretical plates gives a\n'''
    '''distillate that is\n'''
    f'''{(1-chi2)*100:.0f}% iPrOH''',
    xy=(chi2, t2),
    xycoords='data',
    fontsize=12,
    xytext=(+30, -50),
    textcoords='offset points',
    arrowprops={
        'arrowstyle': '->'
    })

In [ ]:
a9 = plt.annotate(
    '''An efficiency of 3\n'''
    '''theoretical plates gives a\n'''
    '''distillate that is\n'''
    f'''{(1-chi3)*100:.0f}% iPrOH''',
    xy=(chi3, t3),
    xycoords='data',
    fontsize=12,
    xytext=(+30, -50),
    textcoords='offset points',
    arrowprops={
        'arrowstyle': '->'
    })

In [ ]:
a10 = plt.annotate(
    '''The greater the number of theoretical plates,\n'''
    '''the more perfect the separation.''',
    xy=(chi4, t5),
    xycoords='data',
    fontsize=12,
    xytext=(+30, -5),
    textcoords='offset points'
    )

In [ ]:
# a1.set_animated(False)
# a1.set_visible(False)

In [ ]:
# a1.set_visible(True)

In [ ]:
# a1.remove()

In [ ]:
from matplotlib.animation import ArtistAnimation

The following define the frames for each step of the animation:

In [ ]:
iproh_bp_frame = [isopropanol_bp_point] + [a_isopropanol_bp]
ibuoh_bp_frame = iproh_bp_frame[:-1] + [isobutanol_bp_point] + [a_isobutanol_bp]
frame_1 = ibuoh_bp_frame[:-1] + [liquid_curve] + [a_liquid]
frame_2 = frame_1[:-1] + [vapor_curve] + [a_vapor]
frame_3 = frame_2[:-1] + [vertical1, l0, a1]
frame_4 = frame_3[:-1] + [horizontal1, v0, a2]
frame_5 = frame_4[:-1] + [vertical2, l1, a3]
frame_5_2 = frame_5[:-1] + [a4]
frame_5_3 = frame_5_2[:-1] + [a5]
frame_5_4 = frame_5_3[:-1] + [a6]
frame_6 = frame_5_3[:-1] + [horizontal2, v1] + [a7]
frame_7 = frame_6[:-1] + [vertical3, l2] + [a8]
frame_8 = frame_7[:-1] + [horizontal3, v2]
frame_9 = frame_8 + [vertical4, l3] + [a9]
frame_10 = frame_9[:-1] + [horizontal4] + [a10]
frame_11 = frame_10 + [vertical5]
frame_12 = frame_11 + [horizontal5]
frame_13 = frame_12 + [vertical6]
frame_14 = frame_13 + [horizontal6]
frame_15 = frame_14 + [vertical7]
intro_frame = [liquid_curve, vapor_curve]
# print (len(frame_1), len(frame_2), len(frame_3), len(frame_4))

In [ ]:
# print(frame_3)

The frames are put in order into the `artists` list:

In [ ]:
artists = [
           intro_frame, iproh_bp_frame, ibuoh_bp_frame, 
           frame_1, frame_2, frame_3, frame_4, 
           frame_5, frame_5_2, frame_5_3, frame_5_4,
           frame_6, frame_7, frame_8, frame_9, frame_10,
           frame_11, frame_12, frame_13, frame_14, frame_15]


In [ ]:
# for artist in artists:
#     print(f'Trying to hide artist {artist}')
#     try:
#         artist.set_visible(False)
#     except Exception as e:
#         print("Doesn't seem to be an artist!")

In [ ]:
# liquid_curve.set_visible(False)

In [ ]:
# print(liquid_curve)

In [ ]:
# Adjust interval (ms) to suit your taste. Here: 5 s per frame
ani = ArtistAnimation(fig, artists, interval=5000)
plt.show()

In [ ]:
# for artist in artists:
#     artist.set_visible(True)

In [ ]:
from IPython.display import HTML

In [ ]:
HTML(ani.to_jshtml())

In [ ]:
print(ani)

In [ ]:
# Uncomment the code below to save the animated gif

# ani.save('distillation_test.gif', writer='imagemagick')
# anim.save('../../files/animation.gif', writer='imagemagick', fps=60)